<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/CNN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login 

     |████████████████████████████████| 1.9 MB 33.4 MB/s 
     |████████████████████████████████| 182 kB 45.4 MB/s 
     |████████████████████████████████| 166 kB 71.0 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 166 kB 66.5 MB/s 
     |████████████████████████████████| 162 kB 30.1 MB/s 
     |████████████████████████████████| 162 kB 78.9 MB/s 
     |████████████████████████████████| 158 kB 70.6 MB/s 
     |████████████████████████████████| 157 kB 72.9 MB/s 
     |████████████████████████████████| 157 kB 73.4 MB/s 
     |████████████████████████████████| 157 kB 61.8 MB/s 
     |████████████████████████████████| 157 kB 73.8 MB/s 
     |████████████████████████████████| 157 kB 74.4 MB/s 
     |████████████████████████████████| 157 kB 79.9 MB/s 
     |████████████████████████████████| 157 kB 73.7 MB/s 
     |████████████████████████████████| 156 kB 78.8 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: h

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras import layers

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

In [6]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width)  )

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.


In [7]:
class_names = train_ds.class_names
test_class_names = test_ds.class_names
class_names == test_class_names
print(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [23]:
num_classes = len(class_names)
data_augmentation = tf.keras.Sequential([tf.keras.layers.RandomFlip("horizontal_and_vertical"),tf.keras.layers.RandomRotation(0.2),tf.keras.layers.RandomZoom(0.1),])
model = Sequential([
  tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  data_augmentation,
  layers.Conv2D(16, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(256, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(180, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(30, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])

In [24]:
model.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [25]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_2 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 sequential_4 (Sequential)   (None, 224, 224, 3)       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 111, 111, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 54, 54, 32)       0         
 g2D)                                                 

In [20]:
'''model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/Research Internship/augmentation/my_best_model.epoch{epoch:02d}-val_accuracy{val_accuracy:.2f}",
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)'''

'model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(\n    filepath="/content/drive/MyDrive/Research Internship/augmentation/my_best_model.epoch{epoch:02d}-val_accuracy{val_accuracy:.2f}",\n    save_weights_only=False,\n    monitor=\'val_accuracy\',\n    mode=\'max\',\n    save_best_only=True)'

In [28]:
wandb.init(
      # Set the project where this run will be logged
      project="CNN_particle_classification", 
      name = "run-5",
)

In [29]:
%%time
epochs=200
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[WandbCallback()]
)

Epoch 1/200
22/22 [==============================] - ETA: 0s - loss: 1.3979 - accuracy: 0.3693

wandb: Adding directory to artifact (/content/wandb/run-20221104_070135-2ypzsiuu/files/model-best)... Done. 0.1s


22/22 [==============================] - 8s 283ms/step - loss: 1.3979 - accuracy: 0.3693 - val_loss: 1.3421 - val_accuracy: 0.3886
Epoch 2/200
22/22 [==============================] - 4s 134ms/step - loss: 1.1570 - accuracy: 0.5142 - val_loss: 1.4451 - val_accuracy: 0.5143
Epoch 3/200
22/22 [==============================] - ETA: 0s - loss: 1.1076 - accuracy: 0.5966

wandb: Adding directory to artifact (/content/wandb/run-20221104_070135-2ypzsiuu/files/model-best)... Done. 0.1s


22/22 [==============================] - 7s 292ms/step - loss: 1.1076 - accuracy: 0.5966 - val_loss: 1.2899 - val_accuracy: 0.5200
Epoch 4/200
22/22 [==============================] - ETA: 0s - loss: 1.0519 - accuracy: 0.6179

...

22/22 [==============================] - 7s 285ms/step - loss: 0.2766 - accuracy: 0.8849 - val_loss: 0.3086 - val_accuracy: 0.8400
Epoch 69/200
22/22 [==============================] - 3s 120ms/step - loss: 0.2860 - accuracy: 0.8963 - val_loss: 0.3409 - val_accuracy: 0.8457
Epoch 70/200
22/22 [==============================] - 3s 121ms/step - loss: 0.2877 - accuracy: 0.8949 - val_loss: 0.5187 - val_accuracy: 0.8000
Epoch 71/200
22/22 [==============================] - 3s 122ms/step - loss: 0.3953 - accuracy: 0.8523 - val_loss: 0.9138 - val_accuracy: 0.7257
Epoch 72/200
22/22 [==============================] - 3s 123ms/step - loss: 0.3559 - accuracy: 0.8608 - val_loss: 0.8406 - val_accuracy: 0.7314
Epoch 73/200
22/22 [==============================] - 3s 122ms/step - loss: 0.3121 - accuracy: 0.8736 - val_loss: 0.3493 - val_accuracy: 0.8400
Epoch 74/200
22/22 [==============================] - 3s 121ms/step - loss: 0.3225 - accuracy: 0.8821 - val_loss: 0.4600 - val_accuracy: 0.8571
Epoch

wandb: Adding directory to artifact (/content/wandb/run-20221104_070135-2ypzsiuu/files/model-best)... Done. 0.1s


22/22 [==============================] - 7s 285ms/step - loss: 0.1897 - accuracy: 0.9219 - val_loss: 0.2200 - val_accuracy: 0.8800
Epoch 91/200
22/22 [==============================] - 3s 125ms/step - loss: 0.2527 - accuracy: 0.9134 - val_loss: 0.3810 - val_accuracy: 0.8800
Epoch 92/200
22/22 [==============================] - 3s 127ms/step - loss: 0.1831 - accuracy: 0.9361 - val_loss: 0.4029 - val_accuracy: 0.8343
Epoch 93/200
22/22 [==============================] - 3s 115ms/step - loss: 0.2539 - accuracy: 0.9347 - val_loss: 0.5640 - val_accuracy: 0.8057
Epoch 94/200
22/22 [==============================] - 3s 115ms/step - loss: 0.3107 - accuracy: 0.8778 - val_loss: 0.7184 - val_accuracy: 0.7829
Epoch 95/200
22/22 [==============================] - 3s 118ms/step - loss: 0.3683 - accuracy: 0.8693 - val_loss: 0.3277 - val_accuracy: 0.8914
Epoch 96/200
22/22 [==============================] - 3s 119ms/step - loss: 0.2835 - accuracy: 0.8949 - val_loss: 0.6252 - val_accuracy: 0.8229
Epoch

wandb: Adding directory to artifact (/content/wandb/run-20221104_070135-2ypzsiuu/files/model-best)... Done. 0.1s


22/22 [==============================] - 6s 267ms/step - loss: 0.1828 - accuracy: 0.9389 - val_loss: 0.1539 - val_accuracy: 0.9429
Epoch 141/200
22/22 [==============================] - 3s 123ms/step - loss: 0.1069 - accuracy: 0.9659 - val_loss: 0.4147 - val_accuracy: 0.8686
Epoch 142/200
22/22 [==============================] - 3s 120ms/step - loss: 0.1321 - accuracy: 0.9602 - val_loss: 0.3434 - val_accuracy: 0.8857
Epoch 143/200
22/22 [==============================] - 3s 114ms/step - loss: 0.1207 - accuracy: 0.9531 - val_loss: 0.3113 - val_accuracy: 0.8914
Epoch 144/200
22/22 [==============================] - 3s 122ms/step - loss: 0.1014 - accuracy: 0.9730 - val_loss: 0.2921 - val_accuracy: 0.9143
Epoch 145/200
22/22 [==============================] - 3s 123ms/step - loss: 0.0930 - accuracy: 0.9702 - val_loss: 0.2207 - val_accuracy: 0.9200
Epoch 146/200
22/22 [==============================] - 3s 114ms/step - loss: 0.1050 - accuracy: 0.9645 - val_loss: 0.3907 - val_accuracy: 0.8857

In [30]:
wandb.finish()

accuracy,▁▃▃▃▄▃▅▅▅▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇██████▇██▇████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▆▆▄▄▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▁▁▂▁▁▁▁
val_accuracy,▁▂▂▂▄▄▅▅▅▆▇▆▆▆▅▆▆▇▆▇▇▅▆▇▇▇▇████▆▆███▇███
val_loss,█▆▆▆▄▅▄▃▃▂▂▃▃▁▄▃▂▂▂▂▁▆▂▁▁▂▂▁▁▁▁▄▃▁▁▁▂▂▁▁
accuracy,0.96307
best_epoch,139
best_val_loss,0.15386
epoch,199
loss,0.13459
val_accuracy,0.92571


In [31]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
